## Notebook-Interface for controlling the preprcessing part

In [5]:
# source /Users/me/Desktop/Code/venvs/open_diabetis/bin/activate


# Link to github repo:


import os
import json
import datetime
import csv

import pandas as pd
import gzip


'''
%load_ext lab_black
%matplotlib inline
'''
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")


from data_io import File_IO, Database_IO
from data_wrangling import  Data_Wrangling
from data_summarization import Data_Summarization

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
file_io = File_IO() # handles input and output operations

data_wrangling = Data_Wrangling(data_io=file_io) # includes all the transformations
data_summarizations = Data_Summarization(data_io=file_io)

# if db not usage not wished, unccoment temporarly
#database_io = Database_IO(
#    host_ip='81.169.210.147', 
#    port=5849, 
#    db_user='postgres', 
#    db_pw='jhg23675sdfjhg276', 
#    db_name='postgres') # use default postgres db

In [10]:
# 1. Read in all json files for each patient

parent_folder = "/mnt/Ext/home/Nextcloud/Charité/Forschung/OPEN Diabetes/OpenHumans-Falk-GIT/OpenHumansDataTools/data" # if not exists create dir before
patients_jsons_dict = file_io.get_dict_of_folder_and_json_files(parent_folder)


In [13]:
# 2. merge all json files in one csv for each patient with columns:: path, is_valid, starttime, value, value_str

outpath = '/mnt/Ext/home/Nextcloud/Charité/Forschung/OPEN Diabetes/OpenHumans-Falk-GIT/OpenHumansDataTools/temp/data' # if not exists create dir before
data_wrangling.group_patients_csv_data(patients_jsons_dict, outpath=outpath)


Merge data for patient 86025410

Merge data for patient 58680324

Merge data for patient 74077367

Merge data for patient 96254963

Merge data for patient 61403732

Merge data for patient 84181797


In [14]:
# 3. use the merged files in predefined csv format calculate some statistics

folder = '/mnt/Ext/home/Nextcloud/Charité/Forschung/OPEN Diabetes/OpenHumans-Falk-GIT/OpenHumansDataTools/temp/data'

file_names = file_io.get_csv_files(folder=folder)

outpath='/mnt/Ext/home/Nextcloud/Charité/Forschung/OPEN Diabetes/OpenHumans-Falk-GIT/OpenHumansDataTools/temp/summarizations'
data_summarizations.summarize_data(csv_files=file_names, outpath=outpath)

/mnt/Ext/home/Nextcloud/Charité/Forschung/OPEN Diabetes/OpenHumans-Falk-GIT/OpenHumansDataTools/bin/Preprocessing/data_summarization.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  analysis_df = analysis_df.append({"path": path, "value_count": value_count, "mean": mean, "max": max_value, "min": min_value, "mean_duration": mean_duration}, ignore_index=True)
/mnt/Ext/home/Nextcloud/Charité/Forschung/OPEN Diabetes/OpenHumans-Falk-GIT/OpenHumansDataTools/bin/Preprocessing/data_summarization.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  analysis_df = analysis_df.append({"path": path, "value_count": value_count, "mean": mean, "max": max_value, "min": min_value, "mean_duration": mean_duration}, ignore_index=True)
/mnt/Ext/home/Nextcloud/Charité/Forschung/OPEN Diabetes/OpenHumans-Falk-GIT/OpenHumansDataTool

In [30]:
# 4. Merge data into one single data frame

inpath='/mnt/Ext/home/Nextcloud/Charité/Forschung/OPEN Diabetes/OpenHumans-Falk-GIT/OpenHumansDataTools/temp/summarizations'

file_paths = file_io.get_csv_files(folder=inpath)
dataframes = []
for file_path in file_paths:
    frame = file_io.read_csv_pandas(file_path)
    dataframes.append(frame)

print(len(dataframes))
mergedDf = data_summarizations.merge_pandas_frames(dataframes)

mergedDf.head(10)

6


,path,value_count,mean,max,min,mean_duration
0,applicationinfo.applicationid,22,0.000000,0.00,0.0,NaN
1,applicationinfo.customremote,22,0.000000,0.00,0.0,NaN
2,applicationinfo.hasgitinfo,22,0.000000,0.00,0.0,NaN
3,applicationinfo.versioncode,22,1500.000000,1500.00,1500.0,NaN
4,applicationinfo.versionname,22,0.000000,0.00,0.0,NaN
5,apsdata.algorithm,2911,0.000000,0.00,0.0,NaN
6,apsdata.autosensdata.ratio,2911,1.029154,1.50,0.7,NaN
7,apsdata.currenttemp.minutesrunning,2365,15.460147,120.00,0.0,7.5
8,apsdata.currenttemp.rate,2911,0.925280,3.14,0.0,5.0
9,apsdata.currenttemp.temp,2911,0.000000,0.00,0.0,5.0


In [34]:
mergedDf.to_csv("/mnt/Ext/home/Nextcloud/Charité/Forschung/OPEN Diabetes/OpenHumans-Falk-GIT/OpenHumansDataTools/temp/test.csv")

In [15]:
# 5. insert data from csv files in new database table

# 1. get csv file paths
# 2. create new db table
# 2b optional check if columns are in all csv files the same as well as the type ...
# 3. insert data from all csv files recognized in the given file path to db

file_paths = file_io.get_csv_files(folder=folder)

for file_path in file_paths:
    frame = file_io.read_csv_pandas(file_path)
    
    
    print(frame.head(5))
    break

                     path    value value_str     starttime  duration isValid  \
0    uploadinfo.utcoffset  3600000       NaN  1.623776e+12       NaN     NaN   
1      uploadinfo.counter        2       NaN  1.623776e+12       NaN     NaN   
2  uploadinfo.fileversion        1       NaN  1.623776e+12       NaN     NaN   
3   treatments.isdeletion    False       NaN  1.623776e+12       NaN    True   
4    treatments.mealbolus    False       NaN  1.623776e+12       NaN    True   

   patient_id  
0    84181797  
1    84181797  
2    84181797  
3    84181797  
4    84181797  


In [ ]:
# 4. TODO: merge the summarizations to one summarization file

In [ ]:
# 5. database setup

# Next TODO s: Creeate postgres DB (Done)
# Implement data io to read/write from to db
# filter and insert data into db
# create indices to optimize performance and optimize docker postgres performance

# provide an interface to access the data


In [ ]:
# 5. Filter relevant data based on txt file path selection and insert in postgres db table





In [ ]:
# 6. use csv file or database to access data

In [ ]:
# 7. transform table in pivoted cleaned and accessable table (in postgres with login)

## Playground

Code snipts here are work in progress.

In [ ]:
# 4. merge the summarizations

In [ ]:
#merged_stats = merge_csv_files(stat_file_names)
#merged_stats

### Playgorund - Chat GPT Snippets

In [ ]:
import numpy as np

aggregation = {
    "value_count": "sum", 
    "mean": lambda x: np.sum(x)/np.sum(merged_stats["value_count"]),
    "max": "max",
    "min": "min",
    "mean_duration": "mean"
}

df_aggregated = merged_stats.groupby("path").agg(aggregation)
df_aggregated

#return df_aggregated

In [ ]:

# group by path and aggregate
out = df.groupby('path').agg({'value_count': 'sum', 'mean': lambda x: sum(x)/sum(df['value_count']),
                              'max': 'max', 'min': 'min', 'mean_duration': 'mean'})

In [ ]:
df = pd.read_csv(file_names[0])

In [ ]:
import numpy as np
import pandas as pd

# define list of file names
csv_files = ['file1.csv', 'file2.csv', 'file3.csv', 'file4.csv']

# create empty dataframe
merged_df = pd.DataFrame()

# loop through files
for csv in csv_files:
    # read csv
    df = pd.read_csv(csv)
    
    # calculate aggregates
    df['mean'] = df['mean'] * df['value_count']
    df['min'] = df['min'] * df['value_count']
    df['max'] = df['max'] * df['value_count']
    df['mean_duration'] = df['mean_duration'] * df['value_count']
    
    # group by path
    df_grouped = df.groupby('path').sum()
    
    # calculate new aggregates
    df_grouped['mean'] = df_grouped['mean'] / df_grouped['value_count']
    df_grouped['min'] = df_grouped['min'] / df_grouped['value_count']
    df_grouped['max'] = df_grouped['max'] / df_grouped['value_count']
    df_grouped['mean_duration'] = df_grouped['mean_duration'] / df_grouped['value_count']
    
    # append to merged dataframe
    merged_df = pd.concat([merged_df, df_grouped])
    
# drop duplicates and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)

# display dataframe
print(merged_df)